In [1]:
import argparse
import math
import os
import time
from functools import partial

import numpy as np
import torch
import visdom

import pyro
import pyro.contrib.examples.multi_mnist as multi_mnist
import pyro.optim as optim
import pyro.poutine as poutine
from components.AIR import AIR, latents_to_tensor
from pyro.contrib.examples.util import get_data_directory
from pyro.infer import SVI, JitTraceGraph_ELBO, TraceGraph_ELBO
from utils.viz import draw_many, tensor_to_objs

## Support functions

In [2]:
def count_accuracy(X, true_counts, air, batch_size):
    assert X.size(0) == true_counts.size(0), 'Size mismatch.'
    assert X.size(0) % batch_size == 0, 'Input size must be multiple of batch_size.'
    counts = torch.LongTensor(3, 4).zero_()
    error_latents = []
    error_indicators = []

    def count_vec_to_mat(vec, max_index):
        out = torch.LongTensor(vec.size(0), max_index + 1).zero_()
        out.scatter_(1, vec.type(torch.LongTensor).view(vec.size(0), 1), 1)
        return out

    for i in range(X.size(0) // batch_size):
        X_batch = X[i * batch_size:(i + 1) * batch_size]
        true_counts_batch = true_counts[i * batch_size:(i + 1) * batch_size]
        z_where, z_pres = air.guide(X_batch, batch_size)
        inferred_counts = sum(z.cpu() for z in z_pres).squeeze().data
        true_counts_m = count_vec_to_mat(true_counts_batch, 2)
        inferred_counts_m = count_vec_to_mat(inferred_counts, 3)
        counts += torch.mm(true_counts_m.t(), inferred_counts_m)
        error_ind = 1 - (true_counts_batch == inferred_counts)
        error_ix = error_ind.nonzero().squeeze()
        error_latents.append(latents_to_tensor((z_where, z_pres)).index_select(0, error_ix))
        error_indicators.append(error_ind)

    acc = counts.diag().sum().float() / X.size(0)
    error_indices = torch.cat(error_indicators).nonzero().squeeze()
    if X.is_cuda:
        error_indices = error_indices.cuda()
    return acc, counts, torch.cat(error_latents), error_indices


# Defines something like a truncated geometric. Like the geometric,
# this has the property that there's a constant difference in log prob
# between p(steps=n) and p(steps=n+1).
def make_prior(k):
    assert 0 < k <= 1
    u = 1 / (1 + k + k**2 + k**3)
    p0 = 1 - u
    p1 = 1 - (k * u) / p0
    p2 = 1 - (k**2 * u) / (p0 * p1)
    trial_probs = [p0, p1, p2]
    # dist = [1 - p0, p0 * (1 - p1), p0 * p1 * (1 - p2), p0 * p1 * p2]
    # print(dist)
    return lambda t: trial_probs[t]


# Implements "prior annealing" as described in this blog post:
# http://akosiorek.github.io/ml/2017/09/03/implementing-air.html

# That implementation does something very close to the following:
# --z-pres-prior (1 - 1e-15)
# --z-pres-prior-raw
# --anneal-prior exp
# --anneal-prior-to 1e-7
# --anneal-prior-begin 1000
# --anneal-prior-duration 1e6

# e.g. After 200K steps z_pres_p will have decayed to ~0.04

# These compute the value of a decaying value at time t.
# initial: initial value
# final: final value, reached after begin + duration steps
# begin: number of steps before decay begins
# duration: number of steps over which decay occurs
# t: current time step


def lin_decay(initial, final, begin, duration, t):
    assert duration > 0
    x = (final - initial) * (t - begin) / duration + initial
    return max(min(x, initial), final)


def exp_decay(initial, final, begin, duration, t):
    assert final > 0
    assert duration > 0
    # half_life = math.log(2) / math.log(initial / final) * duration
    decay_rate = math.log(initial / final) / duration
    x = initial * math.exp(-decay_rate * (t - begin))
    return max(min(x, initial), final)

## Args

In [3]:
parser = argparse.ArgumentParser(description="Pyro AIR example", argument_default=argparse.SUPPRESS)
parser.add_argument('-n', '--num-steps', type=int, default=int(1e8),
                    help='number of optimization steps to take')
parser.add_argument('-b', '--batch-size', type=int, default=64,
                    help='batch size')
parser.add_argument('-lr', '--learning-rate', type=float, default=1e-4,
                    help='learning rate')
parser.add_argument('-blr', '--baseline-learning-rate', type=float, default=1e-3,
                    help='baseline learning rate')
parser.add_argument('--progress-every', type=int, default=1,
                    help='number of steps between writing progress to stdout')
parser.add_argument('--eval-every', type=int, default=0,
                    help='number of steps between evaluations')
parser.add_argument('--baseline-scalar', type=float,
                    help='scale the output of the baseline nets by this value')
parser.add_argument('--no-baselines', action='store_true', default=False,
                    help='do not use data dependent baselines')
parser.add_argument('--encoder-net', type=int, nargs='+', default=[200],
                    help='encoder net hidden layer sizes')
parser.add_argument('--decoder-net', type=int, nargs='+', default=[200],
                    help='decoder net hidden layer sizes')
parser.add_argument('--predict-net', type=int, nargs='+',
                    help='predict net hidden layer sizes')
parser.add_argument('--embed-net', type=int, nargs='+',
                    help='embed net architecture')
parser.add_argument('--bl-predict-net', type=int, nargs='+',
                    help='baseline predict net hidden layer sizes')
parser.add_argument('--non-linearity', type=str,
                    help='non linearity to use throughout')
parser.add_argument('--viz', action='store_true', default=True,
                    help='generate vizualizations during optimization')
parser.add_argument('--viz-every', type=int, default=10,
                    help='number of steps between vizualizations')
parser.add_argument('--visdom-env', default='main',
                    help='visdom enviroment name')
parser.add_argument('--load', type=str,
                    help='load previously saved parameters')
parser.add_argument('--save', type=str,
                    help='save parameters to specified file')
parser.add_argument('--save-every', type=int, default=1e4,
                    help='number of steps between parameter saves')
parser.add_argument('--cuda', action='store_true', default=False,
                    help='use cuda')
parser.add_argument('--jit', action='store_true', default=False,
                    help='use PyTorch jit')
parser.add_argument('-t', '--model-steps', type=int, default=3,
                    help='number of time steps')
parser.add_argument('--rnn-hidden-size', type=int, default=256,
                    help='rnn hidden size')
parser.add_argument('--encoder-latent-size', type=int, default=50,
                    help='attention window encoder/decoder latent space size')
parser.add_argument('--decoder-output-bias', type=float,
                    help='bias added to decoder output (prior to applying non-linearity)')
parser.add_argument('--decoder-output-use-sigmoid', action='store_true',
                    help='apply sigmoid function to output of decoder network')
parser.add_argument('--window-size', type=int, default=28,
                    help='attention window size')
parser.add_argument('--z-pres-prior', type=float, default=0.5,
                    help='prior success probability for z_pres')
parser.add_argument('--z-pres-prior-raw', action='store_true', default=False,
                    help='use --z-pres-prior directly as success prob instead of a geometric like prior')
parser.add_argument('--anneal-prior', choices='none lin exp'.split(), default='none',
                    help='anneal z_pres prior during optimization')
parser.add_argument('--anneal-prior-to', type=float, default=1e-7,
                    help='target z_pres prior prob')
parser.add_argument('--anneal-prior-begin', type=int, default=0,
                    help='number of steps to wait before beginning to anneal the prior')
parser.add_argument('--anneal-prior-duration', type=int, default=100000,
                    help='number of steps over which to anneal the prior')
parser.add_argument('--pos-prior-mean', type=float,
                    help='mean of the window position prior')
parser.add_argument('--pos-prior-sd', type=float,
                    help='std. dev. of the window position prior')
parser.add_argument('--scale-prior-mean', type=float,
                    help='mean of the window scale prior')
parser.add_argument('--scale-prior-sd', type=float,
                    help='std. dev. of the window scale prior')
parser.add_argument('--no-masking', action='store_true', default=False,
                    help='do not mask out the costs of unused choices')
parser.add_argument('--seed', type=int, help='random seed', default=None)
parser.add_argument('-v', '--verbose', action='store_true', default=False,
                    help='write hyper parameters and network architecture to stdout')

_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, help='write hyper parameters and network architecture to stdout', metavar=None)

In [4]:
# vars(parser.parse_args(""))
args = argparse.Namespace(**vars(parser.parse_args("")))

In [5]:
if 'save' in args:
    if os.path.exists(args.save):
        raise RuntimeError('Output file "{}" already exists.'.format(args.save))

if args.seed is not None:
    pyro.set_rng_seed(args.seed)

# Build a function to compute z_pres prior probabilities.
if args.z_pres_prior_raw:
    def base_z_pres_prior_p(t):
        return args.z_pres_prior
else:
    base_z_pres_prior_p = make_prior(args.z_pres_prior)

# Wrap with logic to apply any annealing.
def z_pres_prior_p(opt_step, time_step):
    p = base_z_pres_prior_p(time_step)
    if args.anneal_prior == 'none':
        return p
    else:
        decay = dict(lin=lin_decay, exp=exp_decay)[args.anneal_prior]
        return decay(p, args.anneal_prior_to, args.anneal_prior_begin,
                     args.anneal_prior_duration, opt_step)



## Data loader

In [6]:
def load_data():
    inpath = './air/.data'
    X_np, Y = multi_mnist.load(inpath)
    X_np = X_np.astype(np.float32)
    X_np /= 255.0
    X = torch.from_numpy(X_np)
    # Using FloatTensor to allow comparison with values sampled from
    # Bernoulli.
    counts = torch.FloatTensor([len(objs) for objs in Y])
    return X, counts

In [7]:
X, true_counts = load_data()
X_size = X.size(0)
if args.cuda:
    X = X.cuda()

In [8]:
inpath = './air/.data'
X_np, Y = multi_mnist.load(inpath)
X_np = X_np.astype(np.float32)
X_np /= 255.0
counts = np.asarray([len(objs) for objs in Y])

np.save('./data/pyro-mnist', X_np[counts==1])

## Model

In [9]:
model_arg_keys = ['window_size',
                  'rnn_hidden_size',
                  'decoder_output_bias',
                  'decoder_output_use_sigmoid',
                  'baseline_scalar',
                  'encoder_net',
                  'decoder_net',
                  'predict_net',
                  'embed_net',
                  'bl_predict_net',
                  'non_linearity',
                  'pos_prior_mean',
                  'pos_prior_sd',
                  'scale_prior_mean',
                  'scale_prior_sd']
model_args = {key: getattr(args, key) for key in model_arg_keys if key in args}

In [10]:
air = AIR(
        num_steps=args.model_steps,
        x_size=50,
        use_masking=not args.no_masking,
        use_baselines=not args.no_baselines,
        z_what_size=args.encoder_latent_size,
        use_cuda=args.cuda,
        **model_args
    )

In [11]:
if 'load' in args:
    print('Loading parameters...')
    air.load_state_dict(torch.load(args.load))

## Visualize

In [13]:
if args.viz:
    vis = visdom.Visdom(env=args.visdom_env)
    z, x = air.prior(5, z_pres_prior_p=partial(z_pres_prior_p, 0))
    vis.images(draw_many(x, tensor_to_objs(latents_to_tensor(z))))

Setting up a new session...


## Optimizer

In [14]:
def isBaselineParam(module_name, param_name):
    return 'bl_' in module_name or 'bl_' in param_name

def per_param_optim_args(module_name, param_name):
    lr = args.baseline_learning_rate if isBaselineParam(module_name, param_name) else args.learning_rate
    return {'lr': lr}

adam = optim.Adam(per_param_optim_args)
elbo = JitTraceGraph_ELBO() if args.jit else TraceGraph_ELBO()
svi = SVI(air.model, air.guide, adam, loss=elbo)

In [15]:
t0 = time.time()
examples_to_viz = X[5:10]

for i in range(1, args.num_steps + 1):

    loss = svi.step(X, batch_size=args.batch_size, z_pres_prior_p=partial(z_pres_prior_p, i))

    if args.progress_every > 0 and i % args.progress_every == 0:
        print('i={}, epochs={:.2f}, elapsed={:.2f}, elbo={:.2f}'.format(
            i,
            (i * args.batch_size) / X_size,
            (time.time() - t0) / 3600,
            loss / X_size))

    if args.viz and i % args.viz_every == 0:
        trace = poutine.trace(air.guide).get_trace(examples_to_viz, None)
        z, recons = poutine.replay(air.prior, trace=trace)(examples_to_viz.size(0))
        z_wheres = tensor_to_objs(latents_to_tensor(z))

        # Show data with inferred objection positions.
        vis.images(draw_many(examples_to_viz, z_wheres))
        # Show reconstructions of data.
        vis.images(draw_many(recons, z_wheres))

    if args.eval_every > 0 and i % args.eval_every == 0:
        # Measure accuracy on subset of training data.
        acc, counts, error_z, error_ix = count_accuracy(X, true_counts, air, 1000)
        print('i={}, accuracy={}, counts={}'.format(i, acc, counts.numpy().tolist()))
        if args.viz and error_ix.size(0) > 0:
            vis.images(draw_many(X[error_ix[0:5]], tensor_to_objs(error_z[0:5])),
                       opts=dict(caption='errors ({})'.format(i)))

    if 'save' in args and i % args.save_every == 0:
        print('Saving parameters...')
        torch.save(air.state_dict(), args.save)

i=1, epochs=0.00, elapsed=0.00, elbo=-458.62
i=2, epochs=0.00, elapsed=0.00, elbo=-426.70
i=3, epochs=0.00, elapsed=0.00, elbo=-442.49
i=4, epochs=0.00, elapsed=0.00, elbo=-521.60
i=5, epochs=0.01, elapsed=0.00, elbo=-471.12
i=6, epochs=0.01, elapsed=0.00, elbo=-451.81
i=7, epochs=0.01, elapsed=0.00, elbo=-431.13
i=8, epochs=0.01, elapsed=0.00, elbo=-458.54
i=9, epochs=0.01, elapsed=0.00, elbo=-452.91
i=10, epochs=0.01, elapsed=0.00, elbo=-476.90
i=11, epochs=0.01, elapsed=0.00, elbo=-470.64
i=12, epochs=0.01, elapsed=0.00, elbo=-436.65
i=13, epochs=0.01, elapsed=0.00, elbo=-437.49
i=14, epochs=0.01, elapsed=0.00, elbo=-462.52
i=15, epochs=0.02, elapsed=0.00, elbo=-426.83
i=16, epochs=0.02, elapsed=0.00, elbo=-449.11
i=17, epochs=0.02, elapsed=0.00, elbo=-449.11
i=18, epochs=0.02, elapsed=0.00, elbo=-442.87
i=19, epochs=0.02, elapsed=0.00, elbo=-432.52
i=20, epochs=0.02, elapsed=0.00, elbo=-447.63
i=21, epochs=0.02, elapsed=0.00, elbo=-448.20
i=22, epochs=0.02, elapsed=0.00, elbo=-436.

i=179, epochs=0.19, elapsed=0.01, elbo=-484.58
i=180, epochs=0.19, elapsed=0.01, elbo=-422.66
i=181, epochs=0.19, elapsed=0.01, elbo=-467.68
i=182, epochs=0.19, elapsed=0.01, elbo=-440.76
i=183, epochs=0.20, elapsed=0.01, elbo=-443.74
i=184, epochs=0.20, elapsed=0.01, elbo=-476.68
i=185, epochs=0.20, elapsed=0.01, elbo=-452.16
i=186, epochs=0.20, elapsed=0.01, elbo=-483.77
i=187, epochs=0.20, elapsed=0.01, elbo=-445.77
i=188, epochs=0.20, elapsed=0.01, elbo=-447.61
i=189, epochs=0.20, elapsed=0.01, elbo=-442.19
i=190, epochs=0.20, elapsed=0.01, elbo=-474.08
i=191, epochs=0.20, elapsed=0.01, elbo=-460.32
i=192, epochs=0.20, elapsed=0.01, elbo=-419.84
i=193, epochs=0.21, elapsed=0.01, elbo=-409.08
i=194, epochs=0.21, elapsed=0.01, elbo=-422.12
i=195, epochs=0.21, elapsed=0.01, elbo=-466.93
i=196, epochs=0.21, elapsed=0.01, elbo=-472.78
i=197, epochs=0.21, elapsed=0.01, elbo=-539.12
i=198, epochs=0.21, elapsed=0.01, elbo=-467.74
i=199, epochs=0.21, elapsed=0.01, elbo=-449.24
i=200, epochs

i=355, epochs=0.38, elapsed=0.02, elbo=-438.12
i=356, epochs=0.38, elapsed=0.02, elbo=-513.64
i=357, epochs=0.38, elapsed=0.02, elbo=-488.50
i=358, epochs=0.38, elapsed=0.02, elbo=-474.25
i=359, epochs=0.38, elapsed=0.02, elbo=-441.26
i=360, epochs=0.38, elapsed=0.02, elbo=-458.19
i=361, epochs=0.39, elapsed=0.02, elbo=-480.60
i=362, epochs=0.39, elapsed=0.02, elbo=-444.83
i=363, epochs=0.39, elapsed=0.02, elbo=-455.34
i=364, epochs=0.39, elapsed=0.02, elbo=-452.35
i=365, epochs=0.39, elapsed=0.02, elbo=-493.99
i=366, epochs=0.39, elapsed=0.02, elbo=-495.15
i=367, epochs=0.39, elapsed=0.02, elbo=-484.21
i=368, epochs=0.39, elapsed=0.02, elbo=-492.31
i=369, epochs=0.39, elapsed=0.02, elbo=-492.60
i=370, epochs=0.39, elapsed=0.02, elbo=-477.86
i=371, epochs=0.40, elapsed=0.02, elbo=-515.41
i=372, epochs=0.40, elapsed=0.02, elbo=-517.79
i=373, epochs=0.40, elapsed=0.02, elbo=-518.47
i=374, epochs=0.40, elapsed=0.02, elbo=-507.03
i=375, epochs=0.40, elapsed=0.02, elbo=-496.69
i=376, epochs

i=530, epochs=0.57, elapsed=0.03, elbo=-483.73
i=531, epochs=0.57, elapsed=0.03, elbo=-466.83
i=532, epochs=0.57, elapsed=0.03, elbo=-459.14
i=533, epochs=0.57, elapsed=0.03, elbo=-501.33
i=534, epochs=0.57, elapsed=0.03, elbo=-513.18
i=535, epochs=0.57, elapsed=0.03, elbo=-460.19
i=536, epochs=0.57, elapsed=0.03, elbo=-450.70
i=537, epochs=0.57, elapsed=0.03, elbo=-431.68
i=538, epochs=0.57, elapsed=0.03, elbo=-477.25
i=539, epochs=0.57, elapsed=0.03, elbo=-508.38
i=540, epochs=0.58, elapsed=0.03, elbo=-503.40
i=541, epochs=0.58, elapsed=0.03, elbo=-494.72
i=542, epochs=0.58, elapsed=0.03, elbo=-418.84
i=543, epochs=0.58, elapsed=0.03, elbo=-501.41
i=544, epochs=0.58, elapsed=0.03, elbo=-470.38
i=545, epochs=0.58, elapsed=0.03, elbo=-513.53
i=546, epochs=0.58, elapsed=0.03, elbo=-466.48
i=547, epochs=0.58, elapsed=0.03, elbo=-486.19
i=548, epochs=0.58, elapsed=0.03, elbo=-458.27
i=549, epochs=0.59, elapsed=0.03, elbo=-469.32
i=550, epochs=0.59, elapsed=0.03, elbo=-504.93
i=551, epochs

i=705, epochs=0.75, elapsed=0.04, elbo=-460.42
i=706, epochs=0.75, elapsed=0.04, elbo=-449.78
i=707, epochs=0.75, elapsed=0.04, elbo=-489.63
i=708, epochs=0.76, elapsed=0.04, elbo=-474.87
i=709, epochs=0.76, elapsed=0.04, elbo=-514.59
i=710, epochs=0.76, elapsed=0.04, elbo=-492.52
i=711, epochs=0.76, elapsed=0.04, elbo=-510.79
i=712, epochs=0.76, elapsed=0.04, elbo=-533.36
i=713, epochs=0.76, elapsed=0.04, elbo=-511.04
i=714, epochs=0.76, elapsed=0.04, elbo=-473.72
i=715, epochs=0.76, elapsed=0.04, elbo=-441.91
i=716, epochs=0.76, elapsed=0.04, elbo=-487.82
i=717, epochs=0.76, elapsed=0.04, elbo=-519.29
i=718, epochs=0.77, elapsed=0.04, elbo=-501.70
i=719, epochs=0.77, elapsed=0.04, elbo=-496.86
i=720, epochs=0.77, elapsed=0.04, elbo=-536.88
i=721, epochs=0.77, elapsed=0.04, elbo=-467.89
i=722, epochs=0.77, elapsed=0.04, elbo=-486.21
i=723, epochs=0.77, elapsed=0.04, elbo=-478.35
i=724, epochs=0.77, elapsed=0.04, elbo=-506.59
i=725, epochs=0.77, elapsed=0.04, elbo=-493.19
i=726, epochs

...

i=9099, epochs=9.71, elapsed=0.56, elbo=-596.76
i=9100, epochs=9.71, elapsed=0.56, elbo=-608.67
i=9101, epochs=9.71, elapsed=0.56, elbo=-623.32
i=9102, epochs=9.71, elapsed=0.56, elbo=-614.96
i=9103, epochs=9.71, elapsed=0.56, elbo=-607.88
i=9104, epochs=9.71, elapsed=0.56, elbo=-632.96
i=9105, epochs=9.71, elapsed=0.56, elbo=-610.43
i=9106, epochs=9.71, elapsed=0.56, elbo=-597.02
i=9107, epochs=9.71, elapsed=0.56, elbo=-606.38
i=9108, epochs=9.72, elapsed=0.56, elbo=-619.68
i=9109, epochs=9.72, elapsed=0.56, elbo=-610.73
i=9110, epochs=9.72, elapsed=0.56, elbo=-628.27
i=9111, epochs=9.72, elapsed=0.56, elbo=-624.01
i=9112, epochs=9.72, elapsed=0.56, elbo=-615.28
i=9113, epochs=9.72, elapsed=0.56, elbo=-603.11
i=9114, epochs=9.72, elapsed=0.56, elbo=-601.77
i=9115, epochs=9.72, elapsed=0.56, elbo=-603.65
i=9116, epochs=9.72, elapsed=0.56, elbo=-597.96
i=9117, epochs=9.72, elapsed=0.56, elbo=-594.82
i=9118, epochs=9.73, elapsed=0.56, elbo=-593.48
i=9119, epochs=9.73, elapsed=0.56, elbo=

i=9270, epochs=9.89, elapsed=0.57, elbo=-629.59
i=9271, epochs=9.89, elapsed=0.57, elbo=-611.42
i=9272, epochs=9.89, elapsed=0.57, elbo=-615.42
i=9273, epochs=9.89, elapsed=0.57, elbo=-604.61
i=9274, epochs=9.89, elapsed=0.57, elbo=-619.46
i=9275, epochs=9.89, elapsed=0.57, elbo=-605.63
i=9276, epochs=9.89, elapsed=0.57, elbo=-622.44
i=9277, epochs=9.90, elapsed=0.57, elbo=-619.02
i=9278, epochs=9.90, elapsed=0.57, elbo=-597.17
i=9279, epochs=9.90, elapsed=0.57, elbo=-611.81
i=9280, epochs=9.90, elapsed=0.57, elbo=-604.38
i=9281, epochs=9.90, elapsed=0.57, elbo=-597.49
i=9282, epochs=9.90, elapsed=0.57, elbo=-605.65
i=9283, epochs=9.90, elapsed=0.57, elbo=-604.15
i=9284, epochs=9.90, elapsed=0.57, elbo=-609.81
i=9285, epochs=9.90, elapsed=0.57, elbo=-615.02
i=9286, epochs=9.91, elapsed=0.57, elbo=-590.67
i=9287, epochs=9.91, elapsed=0.57, elbo=-601.24
i=9288, epochs=9.91, elapsed=0.57, elbo=-616.54
i=9289, epochs=9.91, elapsed=0.57, elbo=-624.39
i=9290, epochs=9.91, elapsed=0.57, elbo=

i=9440, epochs=10.07, elapsed=0.58, elbo=-618.83
i=9441, epochs=10.07, elapsed=0.58, elbo=-601.29
i=9442, epochs=10.07, elapsed=0.58, elbo=-592.90
i=9443, epochs=10.07, elapsed=0.58, elbo=-618.08
i=9444, epochs=10.07, elapsed=0.58, elbo=-607.34
i=9445, epochs=10.07, elapsed=0.58, elbo=-607.70
i=9446, epochs=10.08, elapsed=0.58, elbo=-614.42
i=9447, epochs=10.08, elapsed=0.58, elbo=-594.35
i=9448, epochs=10.08, elapsed=0.58, elbo=-612.53
i=9449, epochs=10.08, elapsed=0.58, elbo=-598.20
i=9450, epochs=10.08, elapsed=0.58, elbo=-600.56
i=9451, epochs=10.08, elapsed=0.58, elbo=-621.47
i=9452, epochs=10.08, elapsed=0.58, elbo=-623.32
i=9453, epochs=10.08, elapsed=0.58, elbo=-617.59
i=9454, epochs=10.08, elapsed=0.58, elbo=-605.48
i=9455, epochs=10.09, elapsed=0.58, elbo=-617.84
i=9456, epochs=10.09, elapsed=0.58, elbo=-605.12
i=9457, epochs=10.09, elapsed=0.58, elbo=-611.55
i=9458, epochs=10.09, elapsed=0.58, elbo=-597.42
i=9459, epochs=10.09, elapsed=0.58, elbo=-578.51
i=9460, epochs=10.09

i=9608, epochs=10.25, elapsed=0.59, elbo=-612.57
i=9609, epochs=10.25, elapsed=0.59, elbo=-615.23
i=9610, epochs=10.25, elapsed=0.59, elbo=-598.70
i=9611, epochs=10.25, elapsed=0.59, elbo=-597.31
i=9612, epochs=10.25, elapsed=0.60, elbo=-605.37
i=9613, epochs=10.25, elapsed=0.60, elbo=-610.84
i=9614, epochs=10.25, elapsed=0.60, elbo=-595.57
i=9615, epochs=10.26, elapsed=0.60, elbo=-600.55
i=9616, epochs=10.26, elapsed=0.60, elbo=-597.34
i=9617, epochs=10.26, elapsed=0.60, elbo=-609.16
i=9618, epochs=10.26, elapsed=0.60, elbo=-621.27
i=9619, epochs=10.26, elapsed=0.60, elbo=-615.44
i=9620, epochs=10.26, elapsed=0.60, elbo=-606.25
i=9621, epochs=10.26, elapsed=0.60, elbo=-619.65
i=9622, epochs=10.26, elapsed=0.60, elbo=-586.46
i=9623, epochs=10.26, elapsed=0.60, elbo=-615.70
i=9624, epochs=10.27, elapsed=0.60, elbo=-601.50
i=9625, epochs=10.27, elapsed=0.60, elbo=-603.43
i=9626, epochs=10.27, elapsed=0.60, elbo=-605.17
i=9627, epochs=10.27, elapsed=0.60, elbo=-612.38
i=9628, epochs=10.27

KeyboardInterrupt: 